In [2]:
pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 3.2 MB/s eta 0:00:00


In [4]:
import osmnx as ox
import networkx as nx
import folium

# Define a bounding box for New York City
north, south, east, west = 40.7128 + 0.05, 40.7128 - 0.05, -74.0060 + 0.05, -74.0060 - 0.05

# Download the street network data for New York City
G = ox.graph_from_bbox(north, south, east, west, network_type='all')

# Source and destination coordinates (latitude, longitude) for your chosen points in New York City
source_point = (40.7128, -74.0060)  # Replace with your source coordinates
destination_point = (40.7228, -74.0060)  # Replace with your destination coordinates

# Find the nearest network nodes to the source and destination points
source_node = ox.distance.nearest_nodes(G, source_point[1], source_point[0])
destination_node = ox.distance.nearest_nodes(G, destination_point[1], destination_point[0])

# Find the absolute shortest path
shortest_path = nx.shortest_path(G, source_node, destination_node, weight='length')

# Identify major nodes based on degree centrality
degree_centrality = nx.degree_centrality(G)
major_nodes = [node for node in shortest_path if degree_centrality[node] >= 0.02]  # Adjust the threshold as needed

# Print or use major_nodes as needed
print("Major Nodes:", major_nodes)

# Create a map centered on New York City
m = folium.Map(location=[40.7128, -74.0060], zoom_start=14)

# Plot the absolute shortest path on the map
shortest_path_coordinates = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]
folium.PolyLine(locations=shortest_path_coordinates, color='red', weight=5).add_to(m)

# Plot major nodes on the map
for node in major_nodes:
    node_coordinates = (G.nodes[node]['y'], G.nodes[node]['x'])
    folium.Marker(location=node_coordinates, popup=str(node), icon=folium.Icon(color='green')).add_to(m)

m


Major Nodes: []


Major nodes marked


In [4]:
import osmnx as ox
import networkx as nx
import folium

# Define a bounding box for New York City
north, south, east, west = 40.7128 + 0.05, 40.7128 - 0.05, -74.0060 + 0.05, -74.0060 - 0.05

# Download the street network data for New York City
G = ox.graph_from_bbox(north, south, east, west, network_type='all')

# Source and destination coordinates (latitude, longitude) for your chosen points in New York City
source_point = (40.7128, -74.0060)  # Replace with your source coordinates
destination_point = (40.7228, -74.0060)  # Replace with your destination coordinates

# Find the nearest network nodes to the source and destination points
source_node = ox.distance.nearest_nodes(G, source_point[1], source_point[0])
destination_node = ox.distance.nearest_nodes(G, destination_point[1], destination_point[0])

# Find the absolute shortest path
shortest_path = nx.shortest_path(G, source_node, destination_node, weight='length')

# Create a map centered on New York City
m = folium.Map(location=[40.7128, -74.0060], zoom_start=14)

# Plot the absolute shortest path on the map
shortest_path_coordinates = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]
folium.PolyLine(locations=shortest_path_coordinates, color='red', weight=5).add_to(m)

# Plot nodes along the absolute shortest path with their information
for node in shortest_path:
    node_info = G.nodes[node]
    node_coordinates = (node_info['y'], node_info['x'])
    folium.Marker(location=node_coordinates, popup=f"Node ID: {node}\nStreet: {node_info.get('name', 'N/A')}", icon=folium.Icon(color='green')).add_to(m)

m


This code calculates the degree centrality for each node in the shortest path and then identifies the node with the highest degree centrality (most_centralized_node). It marks this node on the map with a blue marker. Adjust the popup content and marker style as needed.

In [17]:
import osmnx as ox
import networkx as nx
import folium

# Define a bounding box for New York City
north, south, east, west = 40.7128 + 0.05, 40.7128 - 0.05, -74.0060 + 0.05, -74.0060 - 0.05

# Download the street network data for New York City
G = ox.graph_from_bbox(north, south, east, west, network_type='all')

# Source and destination coordinates (latitude, longitude) for your chosen points in New York City
source_point = (40.7128, -74.0060)  # Replace with your source coordinates
destination_point = (40.7228, -74.0060)  # Replace with your destination coordinates

# Find the nearest network nodes to the source and destination points
source_node = ox.distance.nearest_nodes(G, source_point[1], source_point[0])
destination_node = ox.distance.nearest_nodes(G, destination_point[1], destination_point[0])

# Find the absolute shortest path
shortest_path = nx.shortest_path(G, source_node, destination_node, weight='length')

# Calculate degree centrality for all nodes in the graph
degree_centrality = nx.degree_centrality(G)

# Identify the top N nodes with highest degree centrality along the route
top_n_nodes = sorted(shortest_path, key=lambda x: degree_centrality[x], reverse=True)[:3]  # Adjust the number (3) as needed

# Sample scattered ride locations (replace with your actual ride data)
ride_locations = [(40.71, -74.01), (40.72, -74.02), (40.73, -74.00)]

# Specify the distance threshold for ride acceptance
distance_threshold_km = 1.6

# Create a map centered on New York City
m = folium.Map(location=[40.7128, -74.0060], zoom_start=14)

# Plot the absolute shortest path on the map
shortest_path_coordinates = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]
folium.PolyLine(locations=shortest_path_coordinates, color='red', weight=5).add_to(m)

# Plot the most centralized nodes along the route
for node in top_n_nodes:
    node_coordinates = (G.nodes[node]['y'], G.nodes[node]['x'])
    folium.Marker(location=node_coordinates, popup=f"Node ID: {node}\nDegree Centrality: {degree_centrality[node]:.4f}", icon=folium.Icon(color='blue')).add_to(m)

# Match rides to the closest centralized nodes
for ride_location in ride_locations:
    ride_node = ox.distance.nearest_nodes(G, ride_location[1], ride_location[0])
    closest_node = min(top_n_nodes, key=lambda x: nx.shortest_path_length(G, source=ride_node, target=x, weight='length'))

    # Check if the distance to the closest node is within the threshold
    if nx.shortest_path_length(G, source=ride_node, target=closest_node, weight='length') <= distance_threshold_km:
        folium.Marker(location=(ride_location[0], ride_location[1]), popup=f"Ride Location\nDistance to Closest Node: {nx.shortest_path_length(G, source=ride_node, target=closest_node, weight='length'):.2f} km").add_to(m)

m


To match drivers with rides based on proximity to centralized nodes on the driver's route, you can follow these steps:

Identify Centralized Nodes on the Driver's Route:

Use a centrality measure (e.g., degree centrality) to identify the most centralized nodes along the driver's route.
Identify Rides:

Have information about the scattered rides, including their locations.
Calculate Distances:

Calculate the distance between each ride and the centralized nodes on the driver's route.
Match Rides to Drivers:

For each driver, find the closest rides based on the calculated distances.

In [7]:
pip install haversine


In [8]:
pip install folium

In [23]:
import osmnx as ox
import networkx as nx
import folium
import random

# Define a bounding box for New York City
north, south, east, west = 40.7128 + 0.05, 40.7128 - 0.05, -74.0060 + 0.05, -74.0060 - 0.05

# Download the street network data for New York City
G = ox.graph_from_bbox(north, south, east, west, network_type='all')

# Source and destination coordinates (latitude, longitude) for your chosen points in New York City
source_point = (40.7128, -74.0060)  # Replace with your source coordinates
destination_point = (40.7228, -74.0060)  # Replace with your destination coordinates

# Find the nearest network nodes to the source and destination points
source_node = ox.distance.nearest_nodes(G, source_point[1], source_point[0])
destination_node = ox.distance.nearest_nodes(G, destination_point[1], destination_point[0])

# Find the absolute shortest path
shortest_path = nx.shortest_path(G, source_node, destination_node, weight='length')

# Calculate degree centrality for all nodes in the graph
degree_centrality = nx.degree_centrality(G)

# Identify the top N nodes with highest degree centrality along the route
top_n_nodes = sorted(shortest_path, key=lambda x: degree_centrality[x], reverse=True)[:3]  # Adjust the number (3) as needed

# Sample scattered ride locations (replace with your actual ride data)
ride_locations = [(40.71, -74.01), (40.72, -74.02), (40.73, -74.00)]

# Specify the distance threshold for ride acceptance
distance_threshold_km = 1.6

# Create a map centered on New York City
m = folium.Map(location=[40.7128, -74.0060], zoom_start=14)

# Plot the absolute shortest path on the map
shortest_path_coordinates = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]
folium.PolyLine(locations=shortest_path_coordinates, color='red', weight=5).add_to(m)

# Plot the most centralized nodes along the route
for node in top_n_nodes:
    node_coordinates = (G.nodes[node]['y'], G.nodes[node]['x'])
    folium.Marker(location=node_coordinates, popup=f"Node ID: {node}\nDegree Centrality: {degree_centrality[node]:.4f}", icon=folium.Icon(color='blue')).add_to(m)

# Match rides to the closest centralized nodes
for ride_location in ride_locations:
    ride_node = ox.distance.nearest_nodes(G, ride_location[1], ride_location[0])
    closest_node = min(top_n_nodes, key=lambda x: nx.shortest_path_length(G, source=ride_node, target=x, weight='length'))

    # Check if the distance to the closest node is within the threshold
    if nx.shortest_path_length(G, source=ride_node, target=closest_node, weight='length') <= distance_threshold_km:
        folium.Marker(location=(ride_location[0], ride_location[1]), popup=f"Ride Location\nDistance to Closest Node: {nx.shortest_path_length(G, source=ride_node, target=closest_node, weight='length'):.2f} km").add_to(m)

# Generate random ride locations for demonstration purposes
num_rides = 5
ride_locations = [(random.uniform(40.71, 40.73), random.uniform(-74.02, -74.00)) for _ in range(num_rides)]

# Specify the distance threshold for ride acceptance
distance_threshold_km = 1.6

# Create a map centered on New York City
m = folium.Map(location=[40.7128, -74.0060], zoom_start=14)

# Plot the absolute shortest path on the map
shortest_path_coordinates = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]
folium.PolyLine(locations=shortest_path_coordinates, color='red', weight=5).add_to(m)

# Plot the most centralized nodes along the route
for node in top_n_nodes:
    node_coordinates = (G.nodes[node]['y'], G.nodes[node]['x'])
    folium.Marker(location=node_coordinates, popup=f"Node ID: {node}\nDegree Centrality: {degree_centrality[node]:.4f}", icon=folium.Icon(color='blue')).add_to(m)

# Match rides to the closest centralized nodes and mark on the map
accepted_rides = []
for ride_location in ride_locations:
    ride_node = ox.distance.nearest_nodes(G, ride_location[1], ride_location[0])
    closest_node = min(top_n_nodes, key=lambda x: nx.shortest_path_length(G, source=ride_node, target=x, weight='length'))

    # Check if the distance to the closest node is within the threshold
    distance_to_closest_node = nx.shortest_path_length(G, source=ride_node, target=closest_node, weight='length')

    if distance_to_closest_node <= distance_threshold_km:
        accepted_rides.append((ride_location[0], ride_location[1]))
        folium.Marker(location=(ride_location[0], ride_location[1]),
                      popup=f"Accepted\nDistance to Closest Node: {distance_to_closest_node:.2f} km",
                      icon=folium.Icon(color='green')).add_to(m)
    else:
        folium.Marker(location=(ride_location[0], ride_location[1]),
                      popup=f"Rejected\nDistance to Closest Node: {distance_to_closest_node:.2f} km",
                      icon=folium.Icon(color='red')).add_to(m)

# Mark the accepted rides explicitly for carpooling
for accepted_ride_location in accepted_rides:
    folium.Marker(location=accepted_ride_location, popup="Accepted for Carpooling", icon=folium.Icon(color='purple')).add_to(m)

m


In [27]:
import osmnx as ox
import networkx as nx
import folium

# Define a bounding box for New York City
north, south, east, west = 40.7128 + 0.05, 40.7128 - 0.05, -74.0060 + 0.05, -74.0060 - 0.05

# Download the street network data for New York City
G = ox.graph_from_bbox(north, south, east, west, network_type='all')

# Source and destination coordinates (latitude, longitude) for your chosen points in New York City
source_point = (40.7128, -74.0060)  # Replace with your source coordinates
destination_point = (40.7228, -74.0060)  # Replace with your destination coordinates

# Find the nearest network nodes to the source and destination points
source_node = ox.distance.nearest_nodes(G, source_point[1], source_point[0])
destination_node = ox.distance.nearest_nodes(G, destination_point[1], destination_point[0])

# Find the absolute shortest path
shortest_path = nx.shortest_path(G, source_node, destination_node, weight='length')

# Calculate degree centrality for all nodes in the graph
degree_centrality = nx.degree_centrality(G)

# Identify the top N nodes with highest degree centrality along the route
top_n_nodes = sorted(shortest_path, key=lambda x: degree_centrality[x], reverse=True)[:3]  # Adjust the number (3) as needed

# Predefined ride locations
ride_locations = [
    (40.717, -74.008),
    (40.718, -74.009),
    (40.721, -74.005),
    (40.719, -74.003),
    (40.720, -74.007)
]

# Specify the distance threshold for ride acceptance
distance_threshold_km = 1.6

# Create a map centered on New York City
m = folium.Map(location=[40.7128, -74.0060], zoom_start=14)

# Plot the absolute shortest path on the map
shortest_path_coordinates = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in shortest_path]
folium.PolyLine(locations=shortest_path_coordinates, color='red', weight=5).add_to(m)

# Plot the most centralized nodes along the route
for node in top_n_nodes:
    node_coordinates = (G.nodes[node]['y'], G.nodes[node]['x'])
    folium.Marker(location=node_coordinates, popup=f"Node ID: {node}\nDegree Centrality: {degree_centrality[node]:.4f}", icon=folium.Icon(color='blue')).add_to(m)

# Match rides to the closest centralized nodes and mark on the map
accepted_rides = []
for ride_location in ride_locations:
    ride_node = ox.distance.nearest_nodes(G, ride_location[1], ride_location[0])
    closest_node = min(top_n_nodes, key=lambda x: nx.shortest_path_length(G, source=ride_node, target=x, weight='length'))

    # Check if the distance to the closest node is within the threshold
    distance_to_closest_node = nx.shortest_path_length(G, source=ride_node, target=closest_node, weight='length')

    if distance_to_closest_node <= distance_threshold_km:
        accepted_rides.append((ride_location[0], ride_location[1]))
        folium.Marker(location=(ride_location[0], ride_location[1]),
                      popup=f"Accepted\nDistance to Closest Node: {distance_to_closest_node:.2f} km",
                      icon=folium.Icon(color='green', icon='check')).add_to(m)
    else:
        folium.Marker(location=(ride_location[0], ride_location[1]),
                      popup=f"Rejected\nDistance to Closest Node: {distance_to_closest_node:.2f} km",
                      icon=folium.Icon(color='red', icon='times')).add_to(m)

# Mark the accepted rides explicitly for carpooling
for accepted_ride_location in accepted_rides:
    folium.Marker(location=accepted_ride_location, popup="Accepted for Carpooling", icon=folium.Icon(color='purple')).add_to(m)

m
